Step1 connect to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Step2 install packages

In [2]:
!pip install -q transformers==4.31.0

In [3]:
!pip install datasets

In [4]:
!pip install accelerate==0.21.0

In [5]:
!pip install bitsandbytes==0.40.2

In [6]:
!pip install xformers

Step3 generate prefix of prmmpt

In [7]:
import os
#import openai
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)



#Step 4. Generate the Dataset

In [8]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='mof_forllama_engscheme_train.json', split="train")

In [9]:
dataset = dataset.train_test_split(test_size=0.995)
train_data = dataset["train"]
test_data = dataset["test"]

In [10]:
init_prompt = "You are a machine extracting the name_of_mof, mof_description, guest_species, applications. Your response must be in the following format:"

In [11]:
def formatting_func(example):
    text={}
    clean_data2=str(example['output'].replace('}]\n\nEND\n\n','').replace('"',"")[3:])
    clean_data1=str(example['input'].replace('\n\n###\n\n',''))


    text['text']=clean_data1
    text['NER']=clean_data2
    return text
prompt=init_prompt+'\n'

for _ in train_data:


    prompt=prompt+'\n'+'Text: '+str(formatting_func(_)['text'])+'\n'+'Extract information:\n'+str(formatting_func(_)['NER'])+'\n'




In [12]:
print(prompt)

You are a machine extracting the name_of_mof, mof_description, guest_species, applications. Your response must be in the following format:

Text: Porous MOFs supported palladium catalysts for phenol hydrogenation: A comparative study on MIL-101 and MIL-53
Two metal organic frameworks (MOFs), chromium benzenedicarboxylates MIL-101 and MIL-53, have been synthesized and used as the support of palladium catalysts. The palladium catalysts were characterized by XRD, TEM, and CO chemisorption. MIL-101 is highly hydrophilic and beneficial as support for fine Pd nanoparticles with an average size of 2.3nm. Microporous MIL-53 is relatively hydrophobic and larger Pd particles with an average size of 4.3nm were formed on the external surface. Pd/MIL-101 showed better phenol selective hydrogenation activity to cyclohexanone (98%) under mild reaction conditions because of its smaller particle size.
Extract information:
name_of_mof: MIL-101, mof_formula: , mof_description: [], guest_species: [], appl

Step 4. Load the Llama2 Model with low RAM demand

In [13]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"



In [ ]:
!huggingface-cli login
#you have to get permission from meta AI and hugging face!

In [15]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

device_map = {"": 0}


In [16]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Step 5 incorporate text into prompt for NER task

In [21]:


#for _ in test_data:
text=formatting_func(test_data[1])['text']
NER=formatting_func(test_data[1])['NER']
    #print(text)
    #print('---------')
    #print(NER)
prompt_gen=prompt+'\n'+'Text: '+text+'\n'+'Extract information:\n'+'name_of_mof: '



In [23]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(prompt_gen)
print(result[0]['generated_text'])

You are a machine extracting the name_of_mof, mof_description, guest_species, applications. Your response must be in the following format:

Text: Porous MOFs supported palladium catalysts for phenol hydrogenation: A comparative study on MIL-101 and MIL-53
Two metal organic frameworks (MOFs), chromium benzenedicarboxylates MIL-101 and MIL-53, have been synthesized and used as the support of palladium catalysts. The palladium catalysts were characterized by XRD, TEM, and CO chemisorption. MIL-101 is highly hydrophilic and beneficial as support for fine Pd nanoparticles with an average size of 2.3nm. Microporous MIL-53 is relatively hydrophobic and larger Pd particles with an average size of 4.3nm were formed on the external surface. Pd/MIL-101 showed better phenol selective hydrogenation activity to cyclohexanone (98%) under mild reaction conditions because of its smaller particle size.
Extract information:
name_of_mof: MIL-101, mof_formula: , mof_description: [], guest_species: [], appl

In [24]:
print(result[0]['generated_text'].replace(prompt_gen,''))

 rht-MOFs, mof_formula:, mof_description: [], guest_species: [], applications: [gas adsorption sites]

Text: Highly efficient H2S removal by a metal–organic framework with unsaturated Co2+ sites

A series of isostructural metal–organic frameworks (MOFs), i.e., M-MOF-74 (M=Co2+, Mg2+, and Zn2+), were prepared and tested as gas-phase catalysts for CO oxidation reactions. Maintenance of the porous structure of M-MOF-74 after degassing at high temperature and exposure to air for 24h enabled a successful gas-phase catalytic study of CO oxidation. Among the M-MOF-74 materials tested, only Co-MOF-74 exhibited catalytic activity because of the unrivalled Lewis acidic character of the coordinatively unsaturated sites (CUSs) in Co-MOF-74. Moreover, the catalytic activity of Co-MOF-74 was superior to that of previously reported MOF-based CO oxidation catalysts in terms of the light-off temperature and
